In [1]:
import pandas as pd

def construct_search_df(pathToShowcase: str, pathToCoding: str, pathToReadcode: str) -> pd.DataFrame:
    '''Constructs one large dataframe from showcase.csv, codings.csv (downloaded through downloadCurrentShowcaseFiles), and readcodes.csv (provided)
    
    Keyword arguments:
    ------------------
    pathToShowcase: str
        location of showcase.csv
    pathToCoding: str
        location of codings.csv
    pathToReadcode: str
        location of readcodes.csv

    Returns:
    --------
    searchDf: pd.DataFrame
        searchable dataframe with the columns [Field, FieldID, Coding, Value, Meaning,]
    
    '''
    showcase = pd.read_csv(pathToShowcase, dtype=str)
    codings = pd.read_csv(pathToCoding, dtype=str)
    readcodes = pd.read_csv(pathToReadcode, dtype=str)
    showcase_excerpt = showcase[['Field', 'FieldID', 'Coding']]
    
    readcodes = readcodes.query("type == 'read_2' or type == 'read_3'").rename(columns={"type": "Coding", "code": "Value", "description": "Meaning"})
    readcodes['Field'] = 'gp_clinical, ' + readcodes.Coding
    readcodes['FieldID'] = readcodes.Coding
    readcodes = readcodes.drop(["Unnamed: 0"], axis=1)
    
    searchDf = showcase_excerpt.merge(codings, how='outer', on="Coding")
    searchDf = pd.concat([searchDf,readcodes])
    
    searchDf.Coding = searchDf.Coding.astype('str')
    searchDf.FieldID = searchDf.FieldID.astype('str')
        
    return searchDf


def construct_candidate_df(searchDf: pd.DataFrame, searchTerms: list) -> pd.DataFrame:
    '''Search dataframe searchDf for conditions containing searchTerms 
    
    Keyword arguments:
    ------------------
    searchDf: pd.DataFrame
        searchable dataframe with the columns [Field, FieldId, Coding, Value, Meaning]
    searchTerms: list(str)
        conditions to include in the search

    Returns:
    --------
    searchDf: pd.DataFrame
        filtererd dataframe with the columns [Field, FieldID, Coding, Value, Meaning]
    
    '''
    searchTerms = [x.lower() for x in searchTerms]
    
    fields = searchDf.Field.str.lower().str.contains('|'.join(searchTerms), na = False)
    meanings = searchDf.Meaning.str.lower().str.contains('|'.join(searchTerms), na = False)
    searchDf = searchDf[fields].merge(searchDf[meanings], how='outer')
    return searchDf


def select_conditions(searchDf: pd.DataFrame, searchLogic: str = "or") -> dict():
    '''Interactively search dataframe searchDf for conditions containing searchTerms. 
    
    Keyword arguments:
    ------------------
    searchDf: pd.DataFrame
        searchable dataframe with the columns [Field, FieldID, Coding, Value, Meaning]
    searchLogic: str: ["or"], "and", "not"
        
    Returns:
    --------
    searchDict: dict
          dictionary with three keys: and, or, and none.
          each entry holds a list of tuples (Column number, 'gp_clinical, read_2', or 'gp_clinical, read_3' and search code or 'any') to be included in the search query.
          by default, "and" and "none" will be empty.
    '''
    
    codeColors = {
        "default": '\033[0m',
        "bold": '\033[1m'
    }
        
    print(codeColors["bold"] + 'The following fields have potentially relevant values. Please choose if you want to include all patients who have any value in this field [a], none [hit enter], or if you would like to choose specific values [c].' + codeColors["default"])
    choice = []
    fieldTuples = [(searchDf.Field.loc[a], searchDf.FieldID.loc[a]) for a in searchDf.index]
    for field in set(fieldTuples):
        include = input("Include {}? [a/c/_] ".format(field[0]))
        fieldCode = field[1]
        if include == "a" or include == "A":
            choice.append((fieldCode,'any'))
        elif include == "c" or include == "C":
            print(codeColors["bold"] + "Please choose which codes to include [i] or skip entry [hit enter], skip rest of field [s]." + codeColors["default"])
            fieldDf = searchDf[searchDf.Field == field[0]]
            meaningTuples = [(fieldDf.Meaning.loc[a], fieldDf.Value.loc[a]) for a in fieldDf.index]
            for meaning in set(meaningTuples):
                include = input('    Include {}? [i/_/s] '.format(meaning[0]))
                if include == 'i' or include == 'I':
                    choice.append((fieldCode,meaning[1]))
                if include == 's' or include == 'S':
                    break
    searchDict = dict()
    searchDict[searchLogic] = choice
    return searchDict


def update_inclusion_logic(searchDict: dict, searchDf: pd.DataFrame) -> dict:
    '''Interactively update for logical search conditions. 
    
    Keyword arguments:
    ------------------
    searchDict: dict
          dictionary with three keys: and, or, and none.
          each entry holds a list of tuples (Column number, 'gp_clinical, read_2', or 'gp_clinical, read_3' and search code or 'any') to be included in the search query.            
    searchDf: pd.DataFrame
        searchable dataframe with the columns [Field, FieldID, Coding, Value, Meaning]

        
    Returns:
    --------
    searchDict: dict
          dictionary with three keys: and, or, and none.
          each entry holds a list of tuples (Column number, 'gp_clinical, read_2', or 'gp_clinical, read_3' and search code or 'any') to be included in the search query.
   '''
    
    
    codeColors = {
        "default": '\033[0m',
        "bold": '\033[1m'
    }
    print(codeColors['bold'] + 'Please choose if the following conditions are mandatory (each patient in your cohort will have this condition) [m], optional (all patients in your cohort will have one or more of these conditions) [o], or undesired (none of the patients in your cohort will have this condition) [e]')
    returnSearchDict = dict()
    returnSearchDict["and"] = []
    returnSearchDict["or"] = []
    returnSearchDict["not"] = []
    
    for logicKey in searchDict.keys():
        for entry in searchDict[logicKey]:
            field = entry[0]
            value = entry[1]
            print(entry)
            print(field)
            print(value)
            if value != 'any':
                dfRow = searchDf.query('FieldID == "{}" and Value == "{}"'.format(field, value)).iloc[0]
                fieldDescription = dfRow['Field']
                valueDescription = dfRow['Meaning']
            else: 
                dfRow = searchDf.query('FieldID == "{}"'.format(field)).iloc[0]
                fieldDescription = dfRow['Field']
                valueDescription = 'any'

            choice = input("{}, {}".format(fieldDescription, valueDescription))

            if choice == 'm' or choice == "M":
                returnSearchDict['and'].append(entry)
            elif choice == 'o' or choice == "O":
                returnSearchDict['or'].append(entry)
            elif choice == 'e' or choice == "E":
                returnSearchDict['not'].append(entry)

    return returnSearchDict

In [12]:
downloadDirectory='../ukbb-cohort/dataFiles'

pathToShowcase='../ukbb-cohort/dataFiles/showcase.csv'
pathToCoding='../ukbb-cohort/dataFiles/codings.csv'
pathToReadcode='../ukbb-cohort/dataFiles/readcodes.csv'

searchTerms = ['borderline glaucoma']

pathToCredentials = '.'
driverType = 'chrome'
pathToDriver = "going_headless/chromedriver"

In [61]:
searchDf = construct_search_df(pathToShowcase=pathToShowcase, pathToCoding=pathToCoding, pathToReadcode=pathToReadcode)
searchDf = construct_candidate_df(searchDf=searchDf, searchTerms=searchTerms)
searchDict = select_conditions(searchDf=searchDf)

The following fields have potentially relevant values. Please choose if you want to include all patients who have any value in this field [a], none [hit enter], or if you would like to choose specific values [c].


Include Diagnoses - secondary ICD9? [a/c/_]  
Include gp_clinical, read_2? [a/c/_]  c


Please choose which codes to include [i] or skip entry [hit enter], skip rest of field [s].


    Include Borderline glaucoma with anatomical narrow angle? [i/_/s]  i
    Include Borderline glaucoma NOS? [i/_/s]  i
    Include Borderline glaucoma? [i/_/s]  i
    Include Borderline glaucoma steroid responder? [i/_/s]  i
Include gp_clinical, read_3? [a/c/_]  c


Please choose which codes to include [i] or skip entry [hit enter], skip rest of field [s].


    Include Borderline glaucoma? [i/_/s]  i
    Include Borderline glaucoma steroid responder? [i/_/s]  i
    Include (Borderline glaucoma) or (ocular hypertension) or (increased intra-ocular pressure)? [i/_/s]  i
    Include Borderline glaucoma NOS? [i/_/s]  i
    Include Borderline glaucoma? [i/_/s]  i
Include Type of cancer: ICD9? [a/c/_]  
Include Diagnoses - main ICD9? [a/c/_]  
Include Diagnoses - ICD9? [a/c/_]  


In [63]:
searchDict = update_inclusion_logic(searchDict=searchDict, searchDf=searchDf)

Please choose if the following conditions are mandatory (each patient in your cohort will have this condition) [m], optional (all patients in your cohort will have one or more of these conditions) [o], or undesired (none of the patients in your cohort will have this condition) [e]
('read_2', 'F4502')
read_2
F4502


gp_clinical, read_2, Borderline glaucoma with anatomical narrow angle m


('read_3', 'F450z')
read_3
F450z


gp_clinical, read_3, Borderline glaucoma NOS o


('read_3', 'XE18p')
read_3
XE18p


...

gp_clinical, read_3, (Borderline glaucoma) or (ocular hypertension) or (increased intra-ocular pressure) e


('read_3', 'F450.')
read_3
F450.


gp_clinical, read_3, Borderline glaucoma o


('read_3', 'F4503')
read_3
F4503


gp_clinical, read_3, Borderline glaucoma steroid responder o


In [69]:
def _createGpClinicalQueryString(searchDict: dict) -> str:
    """Create search query for UKBB table `gp_clinical` given a searchDict.

    Keyword arguments:
    ------------------
    searchCodeDict: dict
        dictionary that was created using select_conditions or update_inclusion_logic function

    Returns:
    --------
    queryString: str

    The queryString then needs to be copied to the UKBB data website to download a list of relevant eids or used with download_gpclinical()
    """

    queryString = 'SELECT distinct eid FROM gp_clinical WHERE ('

    l = 0
    for logicKey in searchDict.keys():
        e = 0

        if l != 0: 
            queryString += ') AND ('
            
        for entry in searchDict[logicKey]:
            print(entry)

            if entry[0] in ['read_2', 'read_3']:
                field = entry[0]
                code = entry[1]
                if code == 'any':
                    if logicKey != 'not':
                        code = ' IS NOT NULL'
                    else: 
                        code = ' IS NULL'
                else: 
                    if logicKey != 'not':
                        code = " = '{}'".format(code)
                    else:
                        code = " <> '{}'".format(code)
                
                if logicKey == "and": 
                    if e != 0:
                        queryString += ' AND '
                    queryString += field + code 
                    
                elif logicKey == "or":
                    if e != 0:
                        queryString += ' OR '
                    queryString += field + code 
                        
                elif logicKey == "not":
                    if e != 0:
                        queryString += ' AND '
                    queryString += field + code
                e = e+1  
                print("str: " + queryString)
                l += 1
        
    queryString += ')'
    queryString = queryString.replace(' AND ()', '')
    return queryString


In [68]:
_writeSqlQueryString(searchDict)

('read_2', 'F4502')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502'
('read_2', 'F450.')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.'
('read_3', 'F450z')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z'
('read_3', 'XE18p')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z' OR read_3 = 'XE18p'
('read_2', 'F450z')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z' OR read_3 = 'XE18p' OR read_2 = 'F450z'
('read_3', 'F450.')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z' OR read_3 = 'XE18p' OR read_2 = 'F450z' OR read_3 = 'F450.'
('read_3', 'F4503')
str: SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z' OR read_3 = 'XE18p' OR read_2 = 'F450

"SELECT distinct eid FROM gp_clinical WHERE (read_2 = 'F4502' AND read_2 = 'F450.') AND (read_3 = 'F450z' OR read_3 = 'XE18p' OR read_2 = 'F450z' OR read_3 = 'F450.' OR read_3 = 'F4503') AND (read_2 <> 'F4503' AND read_3 <> 'XE18p')"

In [112]:
eids =dict() 

and = 20002: oct and read_2: glaucoma

eids['and'] = (set([1,2,3,4]) and set([3,4,5,6])) 'oct+eyeCondition' everyone with oct

eids['or'] = [3,4,5] + [6] 'any glaucoma'
eids['not'] = [5,6]


In [113]:
set(set(eids['and']) and set(eids['or']))- set(eids['not']) 

{3, 4}

In [77]:
set(set(andeids) and set(oreids))

{3, 4, 5, 6}

In [115]:
# TODO: support manualQueries

def query_databases(searchDict: dict, pathToMain: str, pathToCredentials: str, pathToDriver: str,  driverType: str, includeCodes: bool = True, timeOut: int = 120) -> dict:
    
    eids = dict()
    eids['gp_clinical'] = dict()
    eids['gp_clinical']['and'] = []
    eids['gp_clinical']['or'] = []
    eids['gp_clinical']['not'] = []
    eids['main'] = dict()
    eids['main']['and'] = []
    eids['main']['or'] = []
    eids['main']['not'] = []
    
# reorganise data
    data = dict()
    data['gp_clinical'] = dict()
    data['gp_clinical']['and'] = []
    data['gp_clinical']['or'] = []
    data['gp_clinical']['not'] = []
    
    data['main'] = dict()
    data['main']['and'] = []
    data['main']['or'] = []
    data['main']['not'] = []
            
    for logicKey in searchDict.keys():
        for entry in searchDict[logicKey]:
            column = entry[0]
            code = entry[1]
            if column in ['read_2', 'read_3']:
                data['gp_clinical'][logicKey].append(entry)
            else: 
                data['main'][logicKey].append(entry)
                
                
# call upon the right function to get the eids
    for database in data.keys():
        if len(data[database]) == 0:
            break

        for logicKey in data[database]:
            if len(data[database][logicKey]) == 0:
                break
            
            if database == 'gp_clinical':
                queryString = _create_gpc_query(data[database][logicKey], logicKey)
                
                eids = _query_gpc_data(queryString, pathToCredentials, pathToDriver, driverType, includeCodes, timeOut)
                eids[database][logicKey] = eids
                
            elif database == 'main':
                queryString = _create_mds_query(data[database][logicKey], logicKey)
                
#                 eids = _query_main_data(queryString, pathToMain)
#                 eids[database][logicKey] = eids

    
#     construct set: (set(set(and)+set(and))+set(or))-set(not)
    
    eids = set(set(eids['gp_clinical']['and'] and eids['main']['and']) and (set(eids['gp_clinical']['or']) or set(eids['main']['or']))) - set(set(eids['gp_clinical']['not'] and eids['main']['not']))
    return eids

In [114]:
def _query_gpc_data(queryString: str, pathToCredentials: str, pathToDriver: str,  driverType: str, includeCodes: bool = True, timeOut: int = 120) -> list:
    """Queries UKBB database given a searchCodeDict and returns Eids of matching candidates.

    Keyword arguments:
    ------------------
    queryString: str
        string to query database. most likely created with _create_gpc_query
    pathToCredentials: str
    path to a .py file containing the variables:
    applicationId: str
        ID of the project with UKBB
    username: str
        UKBB user name
    password: str
        UKBB password
    pathToDriver: str
        path to the driver `chromedriver` used by selenium
    driverType: str
        driverType for selenium e.g chrome or firefox
    includeCodes: bool [True]
        set to False to get the inverse set. Useful to create control cohort.
    timeOut: int [120]
        time in seconds to wait for response from UKBB. Useful for more involved queries.

    Returns:
    --------
    eids: list
        List of eids matching the search criterion of the searchCodeDict
    """
    supported_drivers = ['chrome', 'firefox']
    driverType = driverType.lower()

    if not path.exists(pathToCredentials):
        sys.exit("Credentials file not found")

    if driverType not in supported_drivers:
        raise Exception("Program only suports {} drivers, you provided {}. Please install relevant driver and browser. Instructions in README.md".format(supported_drivers, driverType))

    from importlib import import_module
    cred = import_module("credentials", package=pathToCredentials)

#     queryString = _createGpClinicalQueryString(searchCodeDict)
    payload =  download_gpclinical(queryString, cred.applicationId, cred.userName, cred.password, pathToDriver, driverType, timeOut)
    response = _download_data(payload)
    eids = _extract_eids(response)

    if not includeCodes:
        queryString = 'SELECT distinct eid FROM gp_clinical'
        payload =  download_gpclinical(queryString, cred.applicationId, cred.userName, cred.password, pathToDriver, driverType, timeOut)
        response = _download_data(payload)
        alleids = _extract_eids(response)
        eids = list(set(alleids) - set(eids))

    return eids

In [109]:
def _create_gpc_query(entries: tuple, logic) -> str:
    queryString = 'SELECT distinct eid FROM gp_clinical WHERE'
    e = 0
    for entry in entries: 
        field = entry[0]
        code = entry[1]
        if code == 'any':
            if logic != 'not':
                code = ' IS NOT NULL'
            else: 
                code = ' IS NULL'
        else: 
            if logic != 'not':
                code = " = '{}'".format(code)
            else:
                code = " <> '{}'".format(code)

        if logic == "and": 
            if e != 0:
                queryString += ' AND '
            queryString += field + code 

        elif logic == "or":
            if e != 0:
                queryString += ' OR '
            queryString += field + code 

        elif logic == "not":
            if e != 0:
                queryString += ' AND '
            queryString += field + code
        e = e+1  
    return(queryString)
    
    
def _create_mds_query(entries: tuple, logic) -> str:
    return(entries)

In [101]:
searchDict = update_inclusion_logic(searchDict)

{'and': [('read_2', 'F4502'), ('read_2', 'F450.')],
 'or': [('read_3', 'F450z'),
  ('read_3', 'XE18p'),
  ('read_2', 'F450z'),
  ('read_3', 'F450.'),
  ('read_3', 'F4503')],
 'not': [('read_2', 'F4503'), ('read_3', 'XE18p')]}

In [110]:
query_databases(searchDict)

{'gp_clinical': {'and': [('read_2', 'F4502'), ('read_2', 'F450.')], 'or': [('read_3', 'F450z'), ('read_3', 'XE18p'), ('read_2', 'F450z'), ('read_3', 'F450.'), ('read_3', 'F4503')], 'not': [('read_2', 'F4503'), ('read_3', 'XE18p')]}, 'main': {'and': [], 'or': [], 'not': []}}
SELECT distinct eid FROM gp_clinical WHEREread_2 = 'F4502' AND read_2 = 'F450.'
SELECT distinct eid FROM gp_clinical WHEREread_3 = 'F450z' OR read_3 = 'XE18p' OR read_2 = 'F450z' OR read_3 = 'F450.' OR read_3 = 'F4503'
SELECT distinct eid FROM gp_clinical WHEREread_2 <> 'F4503' AND read_3 <> 'XE18p'


{'and': ['doing it'], 'or': ['doing it'], 'not': ['doing it']}

In [ ]:
'any' - include means all: anyone who has a value in that field. 